# Introduccion a redes neuronales y backpropagation - Micrograd

En esta notebook, exploraremos los conceptos fundamentales detrás del cálculo automático de gradientes, un componente esencial para entrenamiento de redes neuronales. Nos basaremos en la primera lección del curso [Neural Networks: Zero to Hero](https://github.com/karpathy/nn-zero-to-hero/tree/master) de Andrej Karpathy y su proyecto [Micrograd](https://github.com/karpathy/micrograd), una implementación simplificada de un motor de autogradiente.

## Objetivos
*   Comenzaremos revisando la definición de la derivada de una función y cómo calcularla numéricamente.
*   Introduciremos la clase `Value`, que representa un escalar y almacena información crucial para backpropagation.
*   A través de ejemplos y visualizaciones, construiremos el Directed Acyclic Graph (DAG) que representa las operaciones.
*   Finalmente, implementaremos el algoritmo de backpropagation.

Al final de esta notebook, tendrás una comprensión sólida de cómo funciona el cálculo de gradientes y su importancia en la optimización de modelos.

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

---
## Derivada de una función

La derivada de una función se calcula como

$$
f'(x) = \lim_{h \to 0} \frac{f(x+h) - f(x)}{h}
$$

Tenemos una función $f(x) = 3x^2 - 4x + 5$

Podemos calcular su derivada en el punto $x=3$ como

$$
f'(3) = \lim_{h \to 0} \frac{f(3+h) - f(3)}{h}
$$

In [ ]:
def f(x):
  return 3*x**2 - 4*x + 5

xs = np.arange(-5, 5, 0.25)
ys = f(xs)
plt.plot(xs, ys)

In [ ]:
# Si h es demasiado cercano a 0, por la limitación de representación de floating point
# el resultado nos va a dar 0, lo que es incorrecto.
h = 0.000001
x = 3
(f(x + h) - f(x))/h

In [ ]:
x0 = 3
y0 = f(3)
m = (f(x0 + h) - f(x0)) / h  # pendiente en 3

def tangent_line(x):
    return m * (x - x0) + y0

# Rango solo para la tangente
xs_tan = np.linspace(1, 4.5, 100)

plt.plot(xs, ys, label="f(x)")
plt.plot(xs_tan, tangent_line(xs_tan), '--', label=f"Tangente en x={x0}")
plt.scatter([x0], [y0], color="red")  # punto de tangencia
plt.legend()
plt.grid(True)
plt.show()


---
## Derivada de una función con más de un valor

Si queremos calcular la derivada de la función $d(a,b,c) = a \cdot b + c$

Tenemos que decidir respecto a que variable vamos a derivar, por ejemplo, si derivamos respecto a $a$, tenemos

$$
\frac{\partial d}{\partial a} = \lim_{h \to 0} \frac{d(a+h,b,c) - d(a,b,c)}{h}
$$



In [ ]:
def d(a, b, c):
  return a*b + c

a = 2.0
b = -3.0
c = 10.0
d(a, b, c)

In [ ]:
h = 0.0001

# inputs
a = 2.0
b = -3.0
c = 10.0

print("Derivada con respecto a 'a'")
d1 = d(a,b,c)
d2 = d(a+h,b,c)

print('d(a, b, c)', d1)
print('d(a, b+h, c)', d2)
print('slope', (d2 - d1)/h)

print("\nDerivada con respecto a 'b'")
d1 = d(a,b,c)
d2 = d(a,b+h,c)

print('d(a, b, c)', d1)
print('d(a, b+h, c)', d2)
print('slope', (d2 - d1)/h)

print("\nDerivada con respecto a 'c'")
d1 = d(a,b,c)
d2 = d(a,b,c+h)

print('d(a, b, c)', d1)
print('d(a, b, c+h)', d2)
print('slope', (d2 - d1)/h)


---
## Construyendo el núcleo de Micrograd

Vamos a crear una clase que contendrá el **valor numérico** asociado a una variable, junto con otros datos útiles que necesitaremos para realizar **Backpropagation**.  

La clase **`Value`** en *Micrograd* no es más que una versión simplificada de la clase **`Tensor`** de [PyTorch](https://pytorch.org/).  
La diferencia principal es que:  
- En **`Value`** solo trabajaremos con **escalares**.  
- En **PyTorch**, en cambio, los tensores pueden tener cualquier dimensión y soportar operaciones mucho más complejas.  

De esta manera, Micrograd nos permite enfocarnos en lo esencial de cómo funciona el cálculo automático de gradientes, sin la complejidad adicional de trabajar con tensores de mayor dimensión.


In [ ]:
class Value:
  def __init__(self, data):
    self.data = data

  def __repr__(self): # Esta función nos permite imprimir el Value de una forma legible
    return f"Value(data={self.data})"

  def __add__(self, other): # La operación suma: Value + Value
    out = Value(self.data + other.data)
    return out

  def __mul__(self, other): # La operación multiplicación: Value * Value
    out = Value(self.data * other.data)
    return out

In [ ]:
a = Value(2.0)
b = Value(-3.0)
c = Value(10.0)
d = a*b + c
d

---
### Creando el Directed Acyclic Graph (DAG)

La clase **`Value`** nos da una estructura básica, pero lo que realmente necesitamos es una forma de **relacionar los valores entre sí**. Queremos poder reconstruir el camino de operaciones que llevaron a cada `Value`, y para eso vamos a usar un **grafo dirigido acíclico (DAG)**.  

En este grafo, cada `Value` conoce:  
- **Los valores de los que proviene** (sus *padres* o *hijos*, según cómo se mire).  
- **La operación** que lo generó.  
- **El gradiente** asociado, que por ahora inicializamos en `0`.  

De esta manera, no solo tenemos los resultados de las operaciones, sino también toda la información necesaria para aplicar **backpropagation** más adelante.


In [ ]:
# Nueva implementación de la clase value
class Value:
  def __init__(self, data, _children=(), _op='', label=''): # Value ahora recibe sus 'hijos' y la operación que se realizó
    self.data = data
    self.grad = 0 # Gradiente inicializado en 0
    self._prev = set(_children)
    self._op = _op
    self.label = label # label nos mejora la visualización de los datos

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other):
    out = Value(self.data + other.data, (self,other), '+') # Guardamos en el nuevo value, los values utilizados y la operación '+'
    return out

  def __mul__(self, other):
    out = Value(self.data * other.data, (self, other), '*') # Guardamos en el nuevo value, los values utilizados y la operación '*'
    return out

**Visualización**

Para visualizar estas nuevas relaciones, vamos a utilizar funciones auxiliares que utilizan [Graphviz](https://graphviz.org/) para crear visualizaciones

In [ ]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right

  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot


In [ ]:
a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')
e = a*b; e.label = 'e' # Agregamos e para tener una Label en la operación a*b
d = e + c; d.label = 'd'
f = Value(-2.0, label='f')
L = d * f; L.label = 'L'
L
draw_dot(L)

---
## Gradiente

¿Qué significa el gradiente en nuestro contexto?  
El gradiente de cada nodo `Value` representa la **derivada de ese valor respecto al nodo final** (en este caso, *L*).

- Para el nodo final (*L*), el gradiente siempre es `1.0`, porque:

$$
\frac{\partial L}{\partial L} = 1
$$

---

### Gradiente en la multiplicación

Tenemos que:

$$
L = d \cdot f
$$

y queremos calcular:

$$
\frac{\partial L}{\partial d}
$$

Usamos la definición de derivada como límite:

$$
\frac{(d+h) \cdot f - d \cdot f}{h} =
\frac{d \cdot f + h \cdot f - d \cdot f}{h} =
\frac{h \cdot f}{h} = f
$$

Entonces:

$$
\frac{\partial L}{\partial d} = f
$$

De manera simétrica:

$$
\frac{\partial L}{\partial f} = d
$$

Esto nos da una **regla general para la multiplicación**:  
- El gradiente con respecto a un factor es el **otro factor**.

### Aplicado a nuestro ejemplo

Si tenemos $d = 4$ y $f = -2$, entonces:  
- El gradiente de $d$ es $f = -2$.  
- El gradiente de $f$ es $d = 4$.  


---
## La regla de la cadena

Ahora nos preguntamos: ¿cómo calculamos los gradientes del **resto de los valores** con respecto a $L$?  

Tomemos el caso de **$c$**. Queremos calcular:

$$
\frac{\partial L}{\partial c}
$$

El problema es que $c$ no está conectado directamente con $L$ en el grafo, sino a través de $d$.  
¿Cómo podemos entonces calcular $\frac{\partial L}{\partial c}$?  
Aplicando la **regla de la cadena**.

En términos generales:

$$
h'(x) = f'(g(x)) \cdot g'(x)
$$

O, dicho de otra forma: si una variable $z$ depende de $y$ y $y$ depende de $x$, entonces:

$$
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial y} \cdot \frac{\partial y}{\partial x}
$$

---

### Aplicado a nuestro ejemplo

En nuestro grafo, $L$ depende de $c$ **a través de $d$**.  
Podemos expresarlo como:

$$
\frac{\partial L}{\partial c} = \frac{\partial L}{\partial d} \cdot \frac{\partial d}{\partial c}
$$

Ya conocemos $\frac{\partial L}{\partial d}$ (lo guardamos en el gradiente de $d$).  
Lo que falta calcular es $\frac{\partial d}{\partial c}$, a lo que llamamos **gradiente local**.

---

### Gradiente en la suma

Sabemos que:

$$
d = c + e
$$

Entonces, derivando respecto a $c$:

$$
\frac{(c+h)+e - (c+e)}{h} =
\frac{c+h+e - c - e}{h} =
\frac{h}{h} = 1.0
$$

Por lo tanto:

$$
\frac{\partial d}{\partial c} = 1.0
$$

Y de manera análoga:

$$
\frac{\partial d}{\partial e} = 1.0
$$


---
---

## Terminando el ejemplo

Ya tenemos:

- $\frac{\partial L}{\partial d} = -2$
- $\frac{\partial d}{\partial c} = 1$
- $\frac{\partial d}{\partial e} = 1$

Con esto, podemos calcular el gradiente de $c$:

$$
\frac{\partial L}{\partial c} =
\frac{\partial L}{\partial d} \cdot \frac{\partial d}{\partial c} =
-2 \cdot 1 = -2
$$

En el caso de la **suma**, el gradiente simplemente **se copia a los operandos**.  
Por lo tanto, el gradiente de $e$ también es:

$$
\frac{\partial L}{\partial e} =
\frac{\partial L}{\partial d} \cdot \frac{\partial d}{\partial e} =
-2 \cdot 1 = -2
$$

---

### Gradientes de $a$ y $b$

Ahora nos faltan los gradientes de $a$ y $b$.  

Sabemos que:

$$
\frac{\partial L}{\partial a} =
\frac{\partial L}{\partial d} \cdot
\frac{\partial d}{\partial e} \cdot
\frac{\partial e}{\partial a}
$$

Pero ya vimos que:

$$
\frac{\partial L}{\partial e} =
\frac{\partial L}{\partial d} \cdot \frac{\partial d}{\partial e} = -2
$$

Así que solo queda calcular $\frac{\partial e}{\partial a}$ (y análogamente para $b$).  
Como $e = a \cdot b$, aplicamos la regla de la multiplicación:

- $\frac{\partial e}{\partial a} = b = -3$
- $\frac{\partial e}{\partial b} = a = 2$

---

### Resultado final

- Gradiente de $a$:

$$
\frac{\partial L}{\partial a} =
\frac{\partial e}{\partial a} \cdot \frac{\partial L}{\partial e} =
(-3) \cdot (-2) = 6
$$

- Gradiente de $b$:

$$
\frac{\partial L}{\partial b} =
\frac{\partial e}{\partial b} \cdot \frac{\partial L}{\partial e} =
(2) \cdot (-2) = -4
$$

---

En resumen:  
- El gradiente se **propaga hacia atrás** siguiendo las conexiones del grafo.  
- En las sumas, el gradiente se copia.  
- En las multiplicaciones, el gradiente se multiplica por el otro factor.  


In [ ]:
# Pongamos los valores calculados en nuestro grafo para visualizarlos
a.grad = 6
b.grad = -4
c.grad = -2
e.grad = -2
d.grad = -2
f.grad = 4
L.grad = 1
draw_dot(L)

---
### ¿Para qué sirve calcular los gradientes?

Teniendo los gradientes de cada variable respecto a $L$, podemos interpretar **cómo cambiaría $L$ si modificamos esas variables**.  
Por ejemplo, si el gradiente de una variable es positivo, aumentar ligeramente su valor hará que $L$ también aumente; si es negativo, al aumentarla, $L$ disminuirá.

Este proceso es la base de cómo funcionan los algoritmos de **optimización** (como *gradient descent*): ajustar las variables en la dirección que reduce la función de pérdida.

⚠️ Importante: este ajuste solo lo hacemos sobre los **nodos hoja**, es decir, aquellos que no dependen de otros y que representan las variables independientes de nuestro modelo.


In [ ]:
a.data += 0.01 * a.grad
b.data += 0.01 * b.grad
c.data += 0.01 * c.grad
f.data += 0.01 * f.grad

# Volvemos a calcular
e = a*b
d = e + c
L = d * f
L

---
## Backpropagation

El proceso que acabamos de realizar es lo que llamamos **Backpropagation**.

Obviamente, calcular Backpropagation manualmente para cáda nodo es ridiculo asi que vamos a ponerle un fin al sufrimiento y automatizemos el proceso.

In [ ]:
# Nueva implementación de la clase value
class Value:
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0
    self._backward = lambda: None # Función que va a propagar el gradiente en cada nodo, por defecto no hace nada
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data},label={self.label})" # Agregamos label para mejorar visualización

  def __add__(self, other):
    out = Value(self.data + other.data, (self,other), '+')

    def _backward(): # Clausura
      self.grad = 1.0 * out.grad
      other.grad = 1.0 * out.grad

    out._backward = _backward # Guardamos la función backward para el nodo suma
    return out

  def __mul__(self, other):
    out = Value(self.data * other.data, (self, other), '*')

    def _backward(): # Clausura
      self.grad = other.data * out.grad
      other.grad = self.data * out.grad

    out._backward = _backward # Guardamos la función backward para el nodo multiplicación
    return out

In [ ]:
a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0, label='f')
L = d * f; L.label = 'L'
L
draw_dot(L)

Ahora podemos automáticamente propagar el gradiente por **un nodo** llamando a su función `backward()`

In [ ]:
# Cuidado! Backward propaga el gradiente a los inputs utilizando el del output, pero al comenzar el gradiente es 0
L.grad = 1.0 # Por tanto, necesitamos setear el gradiente de L = 1
L._backward()
draw_dot(L)

In [ ]:
# Ahora podemos continuar llamando el backward de todos los nodos, en orden
d._backward()
e._backward()
draw_dot(L)

**Extra:** Que pasa si llamamos a `_backward()` de un nodo 'hoja'?

In [ ]:
f._backward()

---
### Implementando `backward` para todo el grafo

La última pieza que nos falta es poder calcular **todos los gradientes** con una sola llamada a `.backward()`.  

La idea es la siguiente: una vez que tenemos la expresión completa en nuestro grafo, queremos recorrerla **de atrás hacia adelante**, es decir, desde el nodo final (*L*) hasta los nodos hoja.  
De esta forma podemos propagar los gradientes a lo largo de todas las operaciones que construyeron el resultado.

Para lograrlo, necesitamos asegurarnos de visitar los nodos en un **orden correcto**, donde cada nodo se procese *después* de todos los nodos de los que depende.  

El algoritmo que nos resuelve este problema se llama  
👉 [**Ordenamiento Topológico (Topological Sort)**](https://es.wikipedia.org/wiki/Ordenamiento_topol%C3%B3gico).  

Con este algoritmo, podemos recorrer el grafo en el orden adecuado y aplicar backpropagation automáticamente para cada nodo.

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fmiro.medium.com%2Fmax%2F1400%2F1*uMg_ojFXts2WZSjcZe4oRQ.png&f=1&nofb=1&ipt=24e248e39237c41398f3f145394cbb5db0ee64a7b00c9ff7ef70a10028b8829a" width="700">

In [ ]:
a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0, label='f')
L = d * f; L.label = 'L'
L
draw_dot(L)

In [ ]:
topo = []
visited = set()

def build_topo(v):
  if v not in visited:
    visited.add(v)
    for child in v._prev:
      build_topo(child) # Continuamos hacia sus inputs
    topo.append(v)

build_topo(L)
topo

In [ ]:
L.grad = 1
for node in reversed(topo):
  node._backward()

In [ ]:
draw_dot(L)

---
Por último, lo que vamos a hacer es mover esta lógica a una función dentro de nuestra clase Value para poder llamarla cómodamente.

In [ ]:
# Nueva implementación de la clase value
class Value:
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data},label={self.label})"

  def __add__(self, other):
    out = Value(self.data + other.data, (self,other), '+')

    def _backward():
      self.grad = 1.0 * out.grad
      other.grad = 1.0 * out.grad

    out._backward = _backward
    return out

  def __mul__(self, other):
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad = other.data * out.grad
      other.grad = self.data * out.grad

    out._backward = _backward
    return out

  def backward(self): # Función backward integrada en la clase value
    topo = []
    visited = set()

    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child) # Continuamos hacia sus inputs
        topo.append(v)

    build_topo(self)

    self.grad = 1
    for node in reversed(topo):
      node._backward()

In [ ]:
a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0, label='f')
L = d * f; L.label = 'L'
L
draw_dot(L)

In [ ]:
L.backward()
draw_dot(L)

---
### Arreglando un pequeño bug de backprop

Nuestro código tiene un bug que ocurre solo en casos específicos, cuando un nodo es utilizado más de una vez en nuestro grafo

In [ ]:
a = Value(3.0, label='a')
b = a+a; b.label = 'b'
b.backward()
draw_dot(b)

Podemos ver que el gradiente de `a` (la derivada de $b$ respecto a `a`) fue calculado como `1` por nuestro algoritmo de backpropagation.  
Sin embargo, la derivada correcta debería ser `2`.

**¿Por que ocurre esto?**

Si miramos nuestro código para la suma, en la operación `a + a` tenemos:
```python
def __add__(self, other):
    out = Value(self.data + other.data, (self,other), '+')

    def _backward():
      self.grad = 1.0 * out.grad
      other.grad = 1.0 * out.grad
```

Aquí `self` y `other` son en realidad el mismo nodo.
Esto significa que primero asignamos `self.grad = out.grad`, y luego lo **sobrescribimos** con `other.grad = out.grad`.
En vez de obtener 2, nos quedamos solo con 1.

Este mismo problema aparece en cualquier operación donde el mismo nodo se use más de una vez, por ejemplo en multiplicaciones como `a * a`.

**¿Cómo lo solucionamos?**

La clave está en que los gradientes no deben sobrescribirse, sino acumularse.
En backpropagation, un nodo puede recibir contribuciones de múltiples caminos del grafo, y todas ellas deben sumarse.




In [ ]:
# Nueva implementación de la clase value
class Value:
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data},label={self.label})"

  def __add__(self, other):
    out = Value(self.data + other.data, (self,other), '+')

    def _backward():
      self.grad += 1.0 * out.grad # Acumular el gradiente
      other.grad += 1.0 * out.grad # Acumular el gradiente

    out._backward = _backward
    return out

  def __mul__(self, other):
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad # Acumular el gradiente
      other.grad += self.data * out.grad # Acumular el gradiente

    out._backward = _backward
    return out

  def backward(self):
    topo = []
    visited = set()

    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)

    build_topo(self)

    self.grad = 1
    for node in reversed(topo):
      node._backward()

In [ ]:
a = Value(3.0, label='a')
b = a+a; b.label = 'b'
b.backward()
draw_dot(b)